In [18]:
import os
import abc
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import matplotlib.pyplot as plt

## Parameters

In [37]:
BINARY_IMAGES_PATH = "binary_images/1000x1000_easy_noise10"
GRAYSCALE_IMAGES_PATH = "grayscale_images/size800_noise10_independent_distance100"

In [38]:
BINARY_IMAGE_OBSEVATION = os.path.join(BINARY_IMAGES_PATH, "image_0_observation.png")
BINARY_IMAGE_GROUND_TRUTH = os.path.join(BINARY_IMAGES_PATH, "image_0_ground_truth.png")
GRAYSCALE_IMAGE_OBSERVATION = os.path.join(GRAYSCALE_IMAGES_PATH, "image_1_observation.png")
GRAYSCALE_IMAGE_GROUND_TRUTH = os.path.join(GRAYSCALE_IMAGES_PATH, "image_1_ground_truth.png")

## Utilities

In [11]:
def load_grayscale_image_as_numpy_array(path):
    return np.asarray(Image.open(path).convert(mode='L')).astype(np.float32)


def load_binary_image_as_numpy_array(path):
    image = load_grayscale_image_as_numpy_array(path)
    return 255.0 * (image > 127.5)


def show_grayscale_image(image):
    plt.figure(figsize=(12, 8))
    plt.tick_params(labelbottom=False, labelleft=False)
    plt.imshow(image, cmap='gray')

In [12]:
def convert_grayscale_image_to_ising_image(input_image):
    return input_image / 127.5 - 1.0


def convert_ising_image_to_grayscale_image(input_image):
    return (input_image + 1.0) * 127.5

In [8]:
def pad_ising_image(input_image):
    return np.pad(input_image, pad_width=1, mode='symmetric')


def unpad_ising_image(input_image):
    return input_image[1:-1, 1:-1]

## Noise reducers

In [4]:
class SupervisedNoiseReducer(object):
    def __init__(self):
        pass

    @abc.abstractmethod
    def _sampler_step(self):
        pass
    
    def reduce_noise(observation, original_image):
        initial_state = pad_ising_image(convert_grayscale_image_to_ising_image(input_image))
        current_state = initial_state.copy()
        for _ in range(iterations_count):
            gibbs_sampler_iteration(initial_state, current_state, coupling_strength)    
        return convert_ising_image_to_grayscale_image(unpad_ising_image(current_state))        
        pass
    

In [ ]:
@functools.lru_cache(maxsize=None)
def get_probability_of_value(value, distribution_mean, distribution_std):
    return scipy.stats.norm.pdf(
        x=value, loc=distribution_mean, scale=distribution_std,
    )

def gibbs_sampler_iteration(initial_state, current_state, coupling_strength):
    row = np.random.randint(low=1, high=(current_state.shape[0] - 1))
    column = np.random.randint(low=1, high=(current_state.shape[1] - 1))
    neighbours_sum = np.sum([
        current_state[row - 1, column], current_state[row + 1, column],
        current_state[row, column - 1], current_state[row, column + 1],
    ])
    positive_coupling = np.exp(coupling_strength * neighbours_sum)
    negative_coupling = np.exp(-coupling_strength * neighbours_sum)
    get_observation_pbty = functools.partial(
        get_probability_of_value,
        value=current_state[row, column],
        distribution_std=1.0,
    )
    positive_potential = get_observation_pbty(distribution_mean=1.0) * positive_coupling
    negative_potential = get_observation_pbty(distribution_mean=-1.0) * negative_coupling
    positive_pbty = positive_potential / (positive_potential + negative_potential)
    if np.random.uniform() <= positive_pbty:
        current_state[row, column] = 1.0
    else:
        current_state[row, column] = -1.0
    

def reduce_noise_using_gibbs_sampler(input_image, coupling_strength, iterations_count):
    initial_state = pad_ising_image(convert_grayscale_image_to_ising_image(input_image))
    current_state = initial_state.copy()
    for _ in range(iterations_count):
        gibbs_sampler_iteration(initial_state, current_state, coupling_strength)    
    return convert_ising_image_to_grayscale_image(unpad_ising_image(current_state))